<a href="https://www.kaggle.com/code/karishmabattina/caloriepred-rforest?scriptVersionId=240919142" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
train_data = pd.read_csv("/kaggle/input/playground-series-s5e5/train.csv")
test_data = pd.read_csv("/kaggle/input/playground-series-s5e5/test.csv")

In [3]:
train_copy, test_copy = train_data, test_data

In [4]:
train_data.sample(5)

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
201620,201620,male,32,169.0,77.0,15.0,95.0,40.3,63.0
135411,135411,female,20,176.0,73.0,13.0,88.0,40.1,49.0
228231,228231,male,35,194.0,98.0,4.0,88.0,38.6,15.0
205205,205205,male,23,169.0,70.0,22.0,111.0,40.6,143.0
405731,405731,male,63,167.0,68.0,21.0,101.0,40.6,138.0


In [5]:
test_data.sample(5)

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp
219103,969103,female,37,172.0,67.0,16.0,96.0,39.9
237122,987122,female,66,169.0,69.0,8.0,87.0,39.9
171295,921295,male,23,182.0,81.0,16.0,94.0,40.4
61042,811042,male,25,177.0,76.0,11.0,88.0,39.5
219082,969082,female,47,157.0,59.0,10.0,86.0,40.2


In [6]:
train_data.shape

(750000, 9)

In [7]:
test_data.shape

(250000, 8)

In [8]:
train_data.isnull().sum()

id            0
Sex           0
Age           0
Height        0
Weight        0
Duration      0
Heart_Rate    0
Body_Temp     0
Calories      0
dtype: int64

In [9]:
test_data.isnull().sum()

id            0
Sex           0
Age           0
Height        0
Weight        0
Duration      0
Heart_Rate    0
Body_Temp     0
dtype: int64

In [10]:
def feature_engineering(df):
    df['bmi'] = df['Weight'] / ((df['Height'] / 100) ** 2)    
    df['exercise_intensity'] = df['Heart_Rate'] / df['Duration']
    df['heart_rate_duration'] = df['Heart_Rate'] * df['Duration']
    df['temp_duration'] = df['Body_Temp'] * df['Duration']
    df['hr_to_temp'] = df['Heart_Rate'] / df['Body_Temp']
    df['hr_to_age'] = df['Heart_Rate'] / df['Age']
    df['age_bmi'] = df['Age'] * df['bmi']
    df['max_heart_rate'] = 220 - df['Age']
    df['heart_rate_intensity'] = df['Heart_Rate'] / df['max_heart_rate']
    return df

In [11]:
train_data = feature_engineering(train_data)
test_data = feature_engineering(test_data)

In [12]:
gender_map = {"male" : 0,"female" : 1}
train_data['Sex'] = train_data['Sex'].replace(gender_map)

train_data=train_data.drop(["id"],axis=1)

/tmp/ipykernel_13/4161382762.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_data['Sex'] = train_data['Sex'].replace(gender_map)


In [13]:
X = train_data.drop(['Calories'], axis=1)
y = train_data['Calories']

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
rf_params = {'n_jobs': -1,
             'n_estimators': 210,
             'min_samples_split': 19,
             'min_samples_leaf': 5,
             'max_features': 'log2',
             'max_depth': 18}

In [17]:
rf_model = RandomForestRegressor(**rf_params)
rf_model.fit(X_train, y_train)
rf_model.score(X_test, y_test)
y_pred = rf_model.predict(X_test)
print(f"RandomForest → R²: {r2_score(y_test, y_pred):.4f}, RMSE: {mean_squared_error(y_test, y_pred, squared=False):.2f}")

RandomForest → R²: 0.9965, RMSE: 3.67


In [18]:
test_data['Sex'] = test_data['Sex'].replace(gender_map)
test_ids = test_data['id']
X_test_final = test_data.drop(["id"], axis=1)


test_predictions = rf_model.predict(X_test_final)

submission = pd.DataFrame({
    'id': test_ids,
    'Calories': test_predictions
})
submission.to_csv('submission.csv', index=False)

/tmp/ipykernel_13/1108437728.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_data['Sex'] = test_data['Sex'].replace(gender_map)


In [19]:
submission

,id,Calories
0,750000,27.504626
1,750001,106.713899
2,750002,88.059918
3,750003,128.465922
4,750004,75.568751
...,...,...
249995,999995,26.204111
249996,999996,9.260192
249997,999997,73.058854
249998,999998,168.912842
